### 1. Setup

In [33]:
import json
import getpass
import os
from tqdm import tqdm
import requests
import time
import uuid
from uuid import uuid4
from pathlib import Path

In [23]:
!pip install langchain_community
!pip install beautifulsoup4 lxml
!pip install unstructured
!pip install -qU "langchain-chroma>=0.1.2"
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.5/27.5 MB 28.2 MB/s eta 0:00:00


In [34]:
import ssl

try:
    _create_unverified_https_context = ssl._create_unverified_context
except AttributeError:
    # Legacy Python that doesn't verify HTTPS certificates by default
    pass
else:
    # Handle target environment that doesn't support HTTPS verification
    ssl._create_default_https_context = _create_unverified_https_context

In [35]:
os.environ["NCP_CLOVASTUDIO_API_KEY"] = getpass.getpass("NCP CLOVA Studio API Key: ")
os.environ["NCP_APIGW_API_KEY"] = getpass.getpass("NCP API Gateway API Key: ")

NCP CLOVA Studio API Key: ··········
NCP API Gateway API Key: ··········


In [36]:
os.environ["NCP_CLOVASTUDIO_APP_ID"] = input("NCP CLOVA Studio App ID: ")

NCP CLOVA Studio App ID: 2c9ce8f0ca6843e49d98ff0ea46efdd2


In [37]:
os.environ["NCP_CLOVASTUDIO_APP_ID_SEGMENTATION"] = input("NCP CLOVA Studio Segmentation App ID: ")

NCP CLOVA Studio Segmentation App ID: 99a5a58972c64c348e16cd21ce5a31f9


### 2. Load

In [38]:
import subprocess

url_to_filename_map = {}

with open("clovastudiourl.txt", "r") as file:
    urls = [url.strip() for url in file.readlines()]

folder_path = "clovastudioguide"

if not os.path.exists(folder_path):
    os.makedirs(folder_path)

for url in urls:
    filename = url.split("/")[-1] + ".html"
    file_path = os.path.join(folder_path, filename)
    subprocess.run(["wget", "--user-agent=RAGCookbook-Crawler/1.0", "-O", file_path, url], check=True)
    url_to_filename_map[url] = filename

with open("url_to_filename_map.json", "w") as map_file:
    json.dump(url_to_filename_map, map_file)

In [39]:
#!pip install beautifulsoup4 lxml #필요시 beautifulsoup4 설치
from langchain_community.document_loaders import BSHTMLLoader

# 폴더 이름에 맞게 수정
html_files_dir = Path('clovastudioguide')

html_files = list(html_files_dir.glob("*.html"))

# 모든 문서 데이터를 저장할 리스트 초기화
clovastudiodatas = []

# 찾은 HTML 파일들에 대해 처리
for html_file in html_files:
    # 각 파일에 대해 BSHTMLLoader 인스턴스 생성
    loader = BSHTMLLoader(str(html_file))
    document_data = loader.load()
    # 로드된 문서 데이터를 리스트에 추가
    clovastudiodatas.append(document_data)
    print(f"Processed {html_file}")

Processed clovastudioguide/how-we-write-rebuttals-dc84742fece1.html


In [40]:
#!pip install unstructured #필요시 unstructured 설치
from langchain_community.document_loaders import UnstructuredHTMLLoader

# 폴더 이름에 맞게 수정
html_files_dir = Path('./clovastudioguide')

html_files = list(html_files_dir.glob("*.html"))

clovastudiodatas = []

for html_file in html_files:
    loader = UnstructuredHTMLLoader(str(html_file))
    document_data = loader.load()
    clovastudiodatas.append(document_data)
    print(f"Processed {html_file}")

Processed clovastudioguide/how-we-write-rebuttals-dc84742fece1.html


In [41]:
with open("url_to_filename_map.json", "r") as map_file:
    url_to_filename_map = json.load(map_file)

filename_to_url_map = {v: k for k, v in url_to_filename_map.items()}

# clovastudiodatas 리스트의 각 Document 객체의 'source' 수정
for doc_list in clovastudiodatas:
    for doc in doc_list:
        extracted_filename = doc.metadata["source"].split("/")[-1]
        if extracted_filename in filename_to_url_map:
            doc.metadata["source"] = filename_to_url_map[extracted_filename]
        else:
            print(f"Warning: {extracted_filename}에 해당하는 URL을 찾을 수 없습니다.")
print(doc.metadata["source"])


https://deviparikh.medium.com/how-we-write-rebuttals-dc84742fece1


In [13]:
# 이중 리스트를 풀어서 하나의 리스트로 만드는 작업
clovastudiodatas_flattened = [item for sublist in clovastudiodatas for item in sublist]

In [14]:
# 처음 3개 문서의 URL과 내용 일부 출력
for doc in clovastudiodatas_flattened[:2]:
    print(f"URL: {doc.metadata['source']}")
    print(f"{doc.page_content[:100]}...")
    print("-" * 50)

URL: https://deviparikh.medium.com/how-we-write-rebuttals-dc84742fece1
Open in app

Sign in

Write

Sign in

How we write rebuttals

Devi Parikh

Follow

10 min read

May ...
--------------------------------------------------


### 3. Chunking

In [42]:
import http.client
from http import HTTPStatus

class CLOVAStudioExecutor:
    def __init__(self, host, request_id=None):
        self._host = host
        self._api_key = os.environ["NCP_CLOVASTUDIO_API_KEY"]
        self._api_key_primary_val = os.environ["NCP_APIGW_API_KEY"]
        self._request_id = request_id or str(uuid.uuid4())

    def _send_request(self, completion_request, endpoint):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request('POST', endpoint, json.dumps(completion_request), headers)
        response = conn.getresponse()
        status = response.status
        result = json.loads(response.read().decode(encoding='utf-8'))
        conn.close()
        return result, status

    def execute(self, completion_request, endpoint):
        res, status = self._send_request(completion_request, endpoint)
        if status == HTTPStatus.OK:
            return res, status
        else:
            error_message = res.get("status", {}).get("message", "Unknown error") if isinstance(res, dict) else "Unknown error"
            raise ValueError(f"오류 발생: HTTP {status}, 메시지: {error_message}")

class SegmentationExecutor(CLOVAStudioExecutor):
    def execute(self, completion_request):
        app_id = os.environ["NCP_CLOVASTUDIO_APP_ID_SEGMENTATION"]
        endpoint = f'/testapp/v1/api-tools/segmentation/{app_id}'
        res, status = super().execute(completion_request, endpoint)
        if status == HTTPStatus.OK and "result" in res:
            return res["result"]["topicSeg"]
        else:
            error_message = res.get("status", {}).get("message", "Unknown error") if isinstance(res, dict) else "Unknown error"
            raise ValueError(f"오류 발생: HTTP {status}, 메시지: {error_message}")

if __name__ == "__main__":
    # 환경 변수가 설정되어 있는지 확인
    required_env_vars = [
        "NCP_CLOVASTUDIO_API_KEY",
        "NCP_APIGW_API_KEY",
        "NCP_CLOVASTUDIO_APP_ID_SEGMENTATION"
    ]

    missing_vars = [var for var in required_env_vars if not os.environ.get(var)]
    if missing_vars:
        raise ValueError(f"Missing required environment variables: {', '.join(missing_vars)}")

    segmentation_executor = SegmentationExecutor(
        host="clovastudio.apigw.ntruss.com"
    )

    chunked_html = []

    for htmldata in tqdm(clovastudiodatas_flattened):
        try:
            request_data = {
                "postProcessMaxSize": 100,
                "alpha": -100,
                "segCnt": -1,
                "postProcessMinSize": -1,
                "text": htmldata.page_content,
                "postProcess": True
            }

            response_data = segmentation_executor.execute(request_data)
            result_data = [' '.join(segment) for segment in response_data]

            for paragraph in result_data:
                chunked_document = {
                    "metadata": htmldata.metadata["source"],
                    "page_content": paragraph
                }
                chunked_html.append(chunked_document)

        except Exception as e:
            print(f"Error processing data from {htmldata.metadata['source']}: {e}")
            # 오류 발생 시 현재 반복을 건너뛰고 다음으로 진행
            continue

    print(len(chunked_html))

100%|██████████| 1/1 [00:04<00:00,  4.78s/it]

113


In [43]:
chunked_html[40]

{'metadata': 'https://deviparikh.medium.com/how-we-write-rebuttals-dc84742fece1',
 'page_content': 'And then give details, describe context, or explain your position.'}

In [44]:
# 청크 분석 및 출력
print(f"\n총 청크 수: {len(chunked_html)}")

# 샘플 청크 출력
print("\n샘플 청크 (처음 3개):")
for i, chunk in enumerate(chunked_html[:3], 1):
    print(f"\n청크 {i}:")
    print(f"메타데이터: {chunk['metadata']}")
    print(f"내용: {chunk['page_content']}")
    print(f"길이: {len(chunk['page_content'])} 문자")



총 청크 수: 113

샘플 청크 (처음 3개):

청크 1:
메타데이터: https://deviparikh.medium.com/how-we-write-rebuttals-dc84742fece1
내용: Open in app Sign in Write Sign in How we write rebuttals Devi Parikh Follow 10 min read May 27, 2020 --
길이: 103 문자

청크 2:
메타데이터: https://deviparikh.medium.com/how-we-write-rebuttals-dc84742fece1
내용: By Devi Parikh, Dhruv Batra, Stefan Lee We frequently find ourselves giving the same advice to different students on how to write rebuttals.
길이: 140 문자

청크 3:
메타데이터: https://deviparikh.medium.com/how-we-write-rebuttals-dc84742fece1
내용: So we thought we’d write it up. Our experience is with AI conferences (e.g., CVPR, ECCV, ICCV, NeurIPS, ICLR, EMNLP).The core guiding principle is that the rebuttal should be thorough, direct, and easy for the Reviewers and Area Chair (RACs) to follow.
길이: 252 문자


### 4. Embedding

In [45]:
# -*- coding: utf-8 -*-

class EmbeddingExecutor:
    def __init__(self, host, api_key, api_key_primary_val, request_id):
        self._host = host
        self._api_key = api_key
        self._api_key_primary_val = api_key_primary_val
        self._request_id = request_id

    def _send_request(self, completion_request):
        headers = {
            'Content-Type': 'application/json; charset=utf-8',
            'X-NCP-CLOVASTUDIO-API-KEY': self._api_key,
            'X-NCP-APIGW-API-KEY': self._api_key_primary_val,
            'X-NCP-CLOVASTUDIO-REQUEST-ID': self._request_id
        }

        conn = http.client.HTTPSConnection(self._host)
        conn.request('POST', '/testapp/v1/api-tools/embedding/v2/2c9ce8f0ca6843e49d98ff0ea46efdd2', json.dumps(completion_request), headers)
        response = conn.getresponse()
        result = json.loads(response.read().decode(encoding='utf-8'))
        conn.close()
        return result

    def execute(self, completion_request):
        res = self._send_request(completion_request)
        if res['status']['code'] == '20000':
            return res['result']['embedding']
        else:
            return 'Error'


In [ ]:
embedding_executor = EmbeddingExecutor(
    host='clovastudio.apigw.ntruss.com',
    api_key= os.environ["NCP_CLOVASTUDIO_API_KEY"],
    api_key_primary_val = os.environ["NCP_APIGW_API_KEY"],
    request_id= os.environ["NCP_CLOVASTUDIO_APP_ID"]
)

request_data = json.loads("""{
"text" : "input text"
}""", strict=False)

response_text = embedding_executor.execute(request_data)
print(request_data)
print(response_text)
print(len(response_text))

{'text': 'input text'}
[-0.9326172, 0.69628906, -0.515625, 0.12683105, -0.29736328, -0.55615234, 0.2388916, 0.15234375, 0.40039062, -0.68359375, 0.29541016, -0.047424316, 0.36669922, -0.81689453, -0.2619629, -0.34375, -0.16601562, 0.36743164, 0.17993164, -0.2705078, -0.80078125, -0.24499512, -0.08093262, 0.34033203, 0.47387695, 0.9682617, 0.2866211, -0.42407227, -0.58154297, -0.20178223, 0.87646484, 0.0181427, -0.7939453, -2.0625, 0.31201172, -0.11657715, -0.12573242, -0.3149414, -1.5751953, 0.27807617, 0.087524414, -0.59716797, 0.55615234, -0.022842407, -0.27807617, -0.081970215, 1.0087891, -0.39575195, -0.7050781, -0.5683594, 0.054107666, 0.52685547, 1.8037109, -0.1784668, -0.31225586, -0.5307617, -0.2388916, 0.16149902, -1.6572266, -0.07635498, -0.2788086, -0.54296875, -0.48706055, -0.27929688, -0.21374512, 1.7119141, 0.91308594, 0.13464355, -0.703125, -0.67089844, -0.33496094, 0.7866211, -0.3413086, -0.08496094, -1.2548828, 1.1201172, 0.6401367, -0.40893555, -0.41479492, 0.6435547,

In [46]:
# ssl
from langchain_community.embeddings import ClovaXEmbeddings
clovax_embeddings = ClovaXEmbeddings(model='bge-m3') # 임베딩 모델을 설정해주세요

text = "클로바 스튜디오"
clovax_embeddings.embed_query(text)

[-0.6166992,
 -0.4165039,
 -2.1074219,
 0.8017578,
 -0.49853516,
 1.3476562,
 -1.3261719,
 0.06732178,
 0.0234375,
 -0.50341797,
 0.55322266,
 -0.27612305,
 0.5678711,
 -0.78564453,
 0.6796875,
 0.038146973,
 -0.8486328,
 -0.6035156,
 0.51123047,
 0.07965088,
 -0.6401367,
 -0.63671875,
 0.11090088,
 -0.5493164,
 1.6416016,
 -0.26098633,
 1.0253906,
 -0.40893555,
 0.33154297,
 -0.8769531,
 -0.6777344,
 -0.57373047,
 -0.020431519,
 -1.1015625,
 0.5830078,
 -1.9179688,
 0.5029297,
 0.86621094,
 -2.4472656,
 0.23706055,
 0.22705078,
 2.2851562,
 0.6088867,
 -0.3737793,
 0.19958496,
 -0.0044174194,
 -0.36547852,
 -0.44262695,
 -0.78222656,
 -0.40551758,
 0.5419922,
 0.56884766,
 0.5102539,
 0.5102539,
 -0.2800293,
 -0.22912598,
 0.0703125,
 -1.3759766,
 -1.7197266,
 0.5546875,
 -0.99902344,
 0.6660156,
 -0.031021118,
 0.21166992,
 0.7011719,
 0.91503906,
 -0.5629883,
 -0.14025879,
 -0.24206543,
 0.4267578,
 -0.77441406,
 -0.3701172,
 0.1204834,
 -0.17248535,
 -1.1767578,
 0.73876953,
 -0.55

In [48]:
# non-ssl
text_json = dict(text=text)
request_data = text_json

response_text = embedding_executor.execute(request_data)
print(request_data)
print(response_text)

NameError: name 'embedding_executor' is not defined

In [49]:
from langchain_core.documents import Document

documents = []

for index, item in enumerate(chunked_html):
    doc = Document(
        page_content=str(item['page_content']),
        metadata={"source": item['metadata']},
        id=str(uuid4())
    )
    documents.append(doc)

In [50]:
# documents 구조 체크
for i, doc in enumerate(documents):
    print(f"Document {i}:")
    print(f"  Page Content: {doc.page_content[:100]}...")  # 첫 100자만 출력
    print(f"  Metadata: {doc.metadata}")
    print(f"  Page Content Type: {type(doc.page_content)}")
    print(f"  Metadata Type: {type(doc.metadata)}")
    print("-" * 40)

Document 0:
  Page Content: Open in app Sign in Write Sign in How we write rebuttals Devi Parikh Follow 10 min read May 27, 2020...
  Metadata: {'source': 'https://deviparikh.medium.com/how-we-write-rebuttals-dc84742fece1'}
  Page Content Type: <class 'str'>
  Metadata Type: <class 'dict'>
----------------------------------------
Document 1:
  Page Content: By Devi Parikh, Dhruv Batra, Stefan Lee We frequently find ourselves giving the same advice to diffe...
  Metadata: {'source': 'https://deviparikh.medium.com/how-we-write-rebuttals-dc84742fece1'}
  Page Content Type: <class 'str'>
  Metadata Type: <class 'dict'>
----------------------------------------
Document 2:
  Page Content: So we thought we’d write it up. Our experience is with AI conferences (e.g., CVPR, ECCV, ICCV, NeurI...
  Metadata: {'source': 'https://deviparikh.medium.com/how-we-write-rebuttals-dc84742fece1'}
  Page Content Type: <class 'str'>
  Metadata Type: <class 'dict'>
----------------------------------------
Docu

### 5. VectorStore

In [51]:
import chromadb
from langchain_chroma import Chroma

# 임베딩 모델 정의
clovax_embeddings = ClovaXEmbeddings(model='bge-m3')

# 로컬 클라이언트 경로 지정
client = chromadb.PersistentClient(path="./chroma_langchain_db") #저장할 로컬 경로

# Chroma 컬렉션 생성
chroma_collection = client.get_or_create_collection(
    name="clovastudiodatas_docs", #collection이 바뀔때마다 이름도 꼭 변경해줘야 합니다.
    metadata={"hnsw:space": "cosine"} #사용하는 임베딩 모델에 따라 ‘l2’, 'ip', ‘cosine’ 중에 사용
)

# Chroma 벡터 저장소 생성
vectorstore = Chroma(
    client=client,
    collection_name="clovastudiodatas_docs",
    embedding_function=clovax_embeddings
)

# tqdm으로 for 루프 감싸기
for doc in tqdm(documents, desc="Adding documents", total=len(documents)):
    # #print(doc)
    # doc_json = dict(text=doc.page_content)
    # #print(doc_json)
    # embeddings = embedding_executor.execute(doc_json)
    # #print(embeddings)
    embeddings = clovax_embeddings.embed_documents([doc.page_content])[0]
    # 문서 추가
    chroma_collection.add(
        ids=[str(uuid.uuid4())],  # 고유한 ID 생성
        documents=[doc.page_content],
        embeddings=[embeddings],
        metadatas=[doc.metadata]
    )
    time.sleep(1.1)  # 이용량 제어를 고려한 1초 이상의 딜레이, 필요에 따라 조정 가능

print("All documents have been added to the vectorstore.")

Adding documents: 100%|██████████| 113/113 [03:11<00:00,  1.69s/it]

All documents have been added to the vectorstore.


In [52]:
import faiss
from langchain_community.vectorstores import FAISS
from langchain_community.docstore.in_memory import InMemoryDocstore

# 임베딩 모델 정의
clovax_embeddings = ClovaXEmbeddings(model='bge-m3')

# FAISS 벡터 저장소 생성
vectorstore_FAISS = FAISS(
    embedding_function=clovax_embeddings,
    index=faiss.IndexFlatIP(1024), # 임베딩 차원 크기
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)

# tqdm으로 for 루프 감싸기
for doc in tqdm(documents, desc="Adding documents", total=len(documents)):
    embeddings = clovax_embeddings.embed_documents([doc.page_content])[0]
    # 문서 추가
    vectorstore_FAISS.add_documents(
        ids=[str(uuid.uuid4())],  # 고유한 ID 생성
        documents=[doc],  # 문자열만 전달
        embeddings=[embeddings]
    )
    time.sleep(1.4)  # 이용량 제어를 고려한 1초 이상의 딜레이, 필요에 따라 조정 가능

print("All documents have been added to the vectorstore.")

Adding documents: 100%|██████████| 113/113 [04:47<00:00,  2.55s/it]

All documents have been added to the vectorstore.


In [25]:
retriever = vectorstore.as_retriever(kwargs={"k": 3})

### 6. PIPELINE

In [29]:
from langchain_community.chat_models import ChatClovaX

llm = ChatClovaX(
    model="HCX-003",
)

In [30]:
from langchain_core.prompts import PromptTemplate

prompt1 = PromptTemplate.from_template(
    """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 원래 가지고있는 지식은 모두 배제하고, 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.

#Question:
{question}

#Context:
{context}

#Answer:"""
)

In [54]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
        """당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 원래 가지고있는 지식은 모두 배제하고, 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.
검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.
"""
    "\n\n"
    "{context}"
)

prompt2 = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{question}"),
    ]
)

In [64]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

# 체인을 생성합니다.
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt2
    | llm
    | StrOutputParser()
)

In [65]:
rag_chain.invoke("How to write rebuttal?")


'To write a rebuttal, follow these steps:\n\n1. Turn your consensus in the sheet into concrete responses in a rebuttal draft. Write concisely but don’t worry about space.\n2. Being convincing and concise is a subtractive process. Write a draft rebuttal. \n\nReference: https://deviparikh.medium.com/how-we-write-rebuttals-dc84742fece1'

In [74]:
results

{'ids': [['e07af85e-fddc-44cb-837b-906262b4021a',
   '6dd420e8-8095-41a8-a595-b7eac261d203',
   'b738232b-2231-4e49-995e-682ad9ea0f0a',
   'ebf1dbf4-246e-4232-9b5f-a25ec385b898',
   '9b07e9e4-601c-487b-91fd-88f9e67c28c1']],
 'embeddings': [array([[-0.20288086,  1.15332019,  0.29248044, ..., -0.58398438,
          -0.59521484,  0.69433588],
         [-0.20288086,  1.15332019,  0.29248044, ..., -0.58398438,
          -0.59521484,  0.69433588],
         [-1.02734375,  1.00292969,  0.61523438, ..., -0.64257812,
          -0.28979489,  0.24365234],
         [-1.02734375,  1.00292969,  0.61523438, ..., -0.64257812,
          -0.28979489,  0.24365234],
         [-0.33959958,  1.20996094,  0.48364258, ..., -0.98193353,
           0.37719727, -0.29711914]])],
 'documents': [['Turn your consensus in the sheet into concrete responses in a rebuttal draft. Write concisely but don’t worry about space.',
   'Turn your consensus in the sheet into concrete responses in a rebuttal draft. Write concisely

In [66]:
from pprint import pprint
from langchain_core.runnables import RunnablePassthrough
from langchain.schema.runnable import RunnableParallel
from langchain_core.output_parsers import StrOutputParser

# 쿼리 텍스트
query_text = "How to write rebuttal"

# 1. 쿼리 임베딩 생성
clovax_embeddings = ClovaXEmbeddings(model='bge-m3')
query_embedding = clovax_embeddings.embed_query(query_text)

# 2. Chroma 컬렉션에 쿼리 실행하여 유사한 문서 검색
results = chroma_collection.query(
    query_embeddings=[query_embedding],
    n_results=5,
    include=["embeddings", "documents", "metadatas", "distances"]
)

# 3. 유사도 검색 결과를 사용하여 리트리버 구성
similarity_retriever = vectorstore.as_retriever(
    kwargs={"k": 5},
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.1, "k": 3}
)

# 4. 검색된 문서를 RAG 체인에 연결하여 답변 생성
rag_chain_from_docs = (
    RunnablePassthrough.assign(context=(lambda x: format_docs(x["context"]))) # because of chain_with_source = RunnableParallel(
    {"context": similarity_retriever, "question": RunnablePassthrough()}
    | prompt2
    | llm
    | StrOutputParser()
)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain_with_source = RunnableParallel(
    {"context": similarity_retriever, "question": RunnablePassthrough()}
).assign(answer=rag_chain_from_docs)


pprint(rag_chain_with_source)

{
  context: VectorStoreRetriever(tags=['Chroma', 'ClovaXEmbeddings'], vectorstore=<langchain_chroma.vectorstores.Chroma object at 0x7ea89756c5e0>, search_type='similarity_score_threshold', search_kwargs={'score_threshold': 0.1, 'k': 3}),
  question: RunnablePassthrough()
}
| RunnableAssign(mapper={
    answer: RunnableAssign(mapper={
              context: RunnableLambda(lambda x: format_docs(x['context']))
            })
            | ChatPromptTemplate(input_variables=['context', 'question'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context'], input_types={}, partial_variables={}, template='당신은 질문-답변(Question-Answering)을 수행하는 친절한 AI 어시스턴트입니다. 당신의 임무는 원래 가지고있는 지식은 모두 배제하고, 주어진 문맥(context) 에서 주어진 질문(question) 에 답하는 것입니다.\n검색된 다음 문맥(context) 을 사용하여 질문(question) 에 답하세요. 만약, 주어진 문맥(context) 에서 답을 찾을 수 없다면, 답을 모른다면 `주어진 정보에서 질문에 대한 정보를 찾을 수 없습니다` 라고 답하세요.\n\n\n{context}'), additional_kwargs={}), HumanMessagePromptT

In [75]:
result = rag_chain_with_source.invoke(query_text)
model_answer=result['answer']
urls = {doc.metadata['source'] for doc in result['context']}

print("- 사용자 질문: {0}\n- 모델의 답변: {1}\n- 참조한 문서의 url: {2}".format(query_text, model_answer, list(urls)))


- 사용자 질문: How to write rebuttal
- 모델의 답변: To write a rebuttal, follow these steps:

1. Read and understand the opposing argument or claim thoroughly. This will help you identify its strengths and weaknesses.

2. Identify any logical fallacies or inaccuracies in the opposing argument. These can be used to counter their claims effectively.

3. Organize your response logically, starting with an introduction that summarizes the opposing argument and outlines your main points. Then, present each point in a clear and concise manner, providing
- 참조한 문서의 url: ['https://deviparikh.medium.com/how-we-write-rebuttals-dc84742fece1']
